# Problem 76 - Counting Summations 

It is possible to write $5$ as a sum in exactly $6$ different ways:
\begin{align*}
    &4 + 1 \\
    &3 + 2 \\
    &3 + 1 + 1 \\
    &2 + 2 + 1 \\
    &2 + 1 + 1 + 1 \\
    &1 + 1 + 1 + 1 + 1 
\end{align*}

How many different ways can $100$ be written as a sum of at least $2$ positive integers?

---



## Theory

### Definition - Partition

According to [Mathematica](https://math.blogoverflow.com/2014/08/25/playing-with-partitions-eulers-pentagonal-theorem/), a tuple $(n_1, n_2, \ldots, n_k)$ of positive integers is a partition of a positive integer $n$ if, 
$$
    n_1 \geq n_2 \geq \ldots \geq n_k 
$$
and 
$$
    n_1 + n_2 + \ldots + n_k = n
$$
If the tuple is a partition of $n$, then each $n_i$ is a part of the partition, $k$ is the number of parts, $n_1$ is the greatest part, and $n_k$ is the least part of the partition.

### Integer partition by coefficients

By considering the 
[combinations](https://math.blogoverflow.com/2014/08/25/playing-with-partitions-eulers-pentagonal-theorem/), 
it follows that if $p(n)$ is the number of paritions of $n$ and $p(0) = 1$ then, 
$$
    \sum_{n=0}^\infty p(n) x^n = \frac{1}{(1-x)(1-x^2)(1-x^3) \cdots} = \prod_{k=1}^\infty \frac{1}{1-x^k}
$$
This form could be used to solve the problem. However, the increasing exponents makes it not a feasible solution. 

### Euler's Pentagonal Theorem

The theorem 
[states](https://math.blogoverflow.com/2014/08/25/playing-with-partitions-eulers-pentagonal-theorem/), 
$$
    (1-x)(1-x^2)(1-x^3) \cdots = \sum_{j=-\infty}^\infty (-1)^j x^{g_j}
$$
where $g_j$ = $\frac{3j^2 + j}{2}$. 
This provides a form from which the partition equation can be simplified.

### Combining Euler's theorem and partition equation

By substituiting Euler's theorem into the partition equation, we obtain,
$$
    \sum_{n=0}^\infty p(n) x^n = \frac{1}{(1-x)(1-x^2)(1-x^3) \cdots} = \frac{1}{\sum_{j=-\infty}^\infty (-1)^j x^{g_j}}
$$
Equating coefficients of $x^n$ yields
[the recurrence relation](https://math.blogoverflow.com/2014/08/25/playing-with-partitions-eulers-pentagonal-theorem/), 
$$
    p(n) = p(n-1) + p(n-2) - p(n-5) - p(n-7) + p(n-12) + p(n-15) - \ldots
$$
where the integers are pentagonal numbers of the form $g_j = (3j^2 \pm j)/2$.

Alternatively, the recursive relation for $p(n)$ can be [expressed as](https://www.cambridge.org/core/journals/lms-journal-of-computation-and-mathematics/article/efficient-implementation-of-the-hardyramanujanrademacher-formula/80A6F53DCB85A4270641920D989A2E8F), 
$$
    p(n) = \sum_{k=1}^n (-1)^{k+1} \Big[ p \Big(n - \frac{k(3k-1)}{2} \Big) + p \Big(n - \frac{k(3k+1)}{2} \Big) \Big ] 
$$

<!-- 
The partition of integers can be described by the recurrence relation,
\begin{gather}
    p_n = \sum_{k=1}^{n} p_k(n) \\
    p_k(n) = p_k(n-k) + p_{k-1}(n-1) \\
    p(n) = \sum_{k \in \mathcal{z}} (-1)^{k+1} p(n-k(3k-1)/2)
\end{gather} -->
<!-- https://www.whitman.edu/mathematics/cgt_online/book/section03.03.html -->
<!-- https://en.wikipedia.org/wiki/Partition_function_(number_theory) -->

## Implementation - brute force

In [2]:
fn pure_recursive_partition_function(input_num: i32) -> i32 
{
    // Put lowest cases into match 
    match input_num {
        0 | 1 => return 1,
        2 => return 2,
        3 => return 3,
        4 => return 5,
        5 => return 7,
        6 => return 11,
        num if num < 0 => return 0,
        _ => ()
    };
    
    let mut sum = 0;
    let mut k = 1;
    
    loop {
        let x: i32 = (k*(3*k - 1))/2;
        
        if x > input_num + k {
            return sum;
        }
        
//         println!("input num = {} {} {}",input_num, input_num - x, input_num -x -k);
        
        let y = pure_recursive_partition_function(input_num - x) + pure_recursive_partition_function(input_num - x - k);
        
        if (k+1) & 1 == 1 {
            sum -= y
        } else {
            sum += y
        }
        
        k += 1;
    }
}

### Performance 

For $p(40)$, it takes approximately $1.77s$. 
This will take a couple of minutes (or more) for $p(100)$.
Therefore, a more efficient solution is sought.

In [3]:
let n = 40;
let soln = pure_recursive_partition_function(n);
println!("p({}) = {}",n, soln);

p(40) = 37338


## Implementation - brute force but store previous value

In [4]:
fn higher_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num + num)/2) as u32
}

fn lower_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num - num)/2) as u32
}



### Maximum $k$ value for a given $n$

Since $p(n) = 0$ for $n < 0$, this can be used to limit the number of iterations and yields, 
$$
    n - g_k = n - (3k^2 \pm k)/2 < 0 
$$
Simply the inequality by considering $=0$ instead of $<0$, 
$$
    3k^2 \mp k - 2n = 0 
$$
Using the quadratic equation, one yields, 
$$
    k = \frac{1}{6} \Big( \pm 1 \pm \sqrt{1+24n} \Big)
$$
$k$ would be largest when the positive case is taken at each $\pm$.

In [5]:
fn maximum_k_val(input_num: u32) -> u32 {
    ((1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
}

### Determine if all prior integers are required

Inspecting the recurrence relation indicates that it is likely that all prior numbers are required to find $p(n)$. 
To verify it, the `required_integers()` function finds which are the integer that are required to assemble $p(n)$.

It was found that this is indeed true. Please refer to three cells down after the function declarations. 

In [6]:
fn compute_pentagonal_num(input_num: u32) -> Vec<u32> {
    let max_k = maximum_k_val(input_num);
    
    if max_k == 0 {
        return Vec::<u32>::new();
    }
    
    (1..=max_k).map(|k| higher_pentagonal_num(k))
                .into_iter()
                .chain(
                    (1..=max_k).map(|k| lower_pentagonal_num(k))
                    .into_iter())
                .collect()
}

In [7]:
fn compute_previous_indices(input_num: u32) -> Vec<(i32, i32)> {
    let max_k = maximum_k_val(input_num);
    
    let casted_input_num = input_num as i32;
    
    if max_k == 0 {
        return Vec::<(i32, i32)>::new();
    }
    
    (1..=max_k).map(|k| 
        (casted_input_num - higher_pentagonal_num(k) as i32, 
            casted_input_num - lower_pentagonal_num(k) as i32))
                .collect()
}

In [8]:
use std::collections::HashSet;

fn compute_integers_from_pent_num(input_num: u32, pentagonal_nums: Vec<u32>) -> HashSet<u32> {
    
    let mut set = HashSet::with_capacity(input_num as usize);
    
    for pent_num in pentagonal_nums.iter() {
        let mut inner_counter = 1;
        
        loop {
            let n_time_k = inner_counter*pent_num;
            
            if n_time_k >= input_num {
                break
            } else {
                set.insert(input_num - n_time_k);
                inner_counter += 1;
            }
        }
    }
    
    set.shrink_to_fit();
    set
}

In [9]:
fn required_integers(input_num: u32) -> Vec<u32> {
    
    let pentagonal_nums = compute_pentagonal_num(input_num);
    println!("Pentagonal Numbers for {}: {:?}", input_num, pentagonal_nums);
    
    let set = compute_integers_from_pent_num(input_num, pentagonal_nums);
    
    let mut out = set.into_iter().collect::<Vec<_>>();
    out.sort();
    out
}

In [10]:
let n = 100;
let prior_integers = required_integers(n);
println!("Prior p(n) that need to be found for {} are: {:?}", n, prior_integers);

Pentagonal Numbers for 100: [2, 7, 15, 26, 40, 57, 77, 100, 1, 5, 12, 22, 35, 51, 70, 92]
Prior p(n) that need to be found for 100 are: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


### Compute values

This utilises the recurrence relation to find $p(n)$.
A more efficient solution would've been to use a vector than a hash map. 
However, it was only realised later that this was not necessary.

In [11]:
use std::collections::HashMap;

fn recursive_partition_function(input_num: u32, storage_map: &HashMap<u32, u32>) -> HashMap<u32, u32>
{
    let mut out = HashMap::with_capacity(1 + storage_map.len());
    
    if input_num < 7 {
        // Put lowest cases into match to reduce computational time
        let out_val = match input_num {0 | 1 => 1,
                                        2 => 2,
                                        3 => 3,
                                        4 => 5,
                                        5 => 7,
                                        6 => 11,
                                        _ => 0
        };
        out.insert(input_num, out_val);
        return out;
    }
    
    let max_k = maximum_k_val(input_num);
    let mut sum: i32 = 0;
    
    for k in 1..=max_k {
        let upper_val: i32 = input_num as i32 - lower_pentagonal_num(k) as i32;
        let lower_val: i32 = input_num as i32 - higher_pentagonal_num(k) as i32;
        
        let vals = [upper_val, lower_val];
//         println!("max k = {}, vals={:?}", max_k, vals);
        
        let mut partition_vals = vec![0; 2];
        
        for (counter, value) in vals.iter().enumerate() {
            
            if value == &0 {
                partition_vals[counter] = 1;
                continue
            } else if value < &0 {
                partition_vals[counter] = 0;
                continue
            }
            
            let val: u32 = *value as u32;
            
            let map_val = storage_map.get(&val);
            
            let num_partitions: u32;
            
            if map_val.is_none() && out.get(&val).is_none() {
                let sub_partition_val = pure_recursive_partition_function(*value as i32) as u32;
                num_partitions = sub_partition_val;
                out.insert(val, sub_partition_val);
            } else if out.get(&val).is_some() {
                num_partitions = *out.get(&val).unwrap();
            } else {
                num_partitions = *map_val.unwrap();
            }
            
            partition_vals[counter] = num_partitions;
        }
        
        
        let temp = (partition_vals[0] + partition_vals[1]) as i32;
        
//         println!("num={}, k={}, pval={:?}, psum={}", input_num, k, partition_vals, temp);
        
        if (k+1) & 1 == 1 {
            sum -= temp
        } else {
            sum += temp
        }
    }
    
    out.insert(input_num, sum as u32);
    
    out.extend(storage_map);
    
    out
}

In [12]:
fn build_base_partition_vals(partition_integers: Vec<u32>) -> HashMap<u32, u32> {
    
    let mut all_partition_vals: HashMap<u32, u32> = HashMap::new();
    
    for val in partition_integers.iter() {
        all_partition_vals = recursive_partition_function(*val, &all_partition_vals);
    }
    
    all_partition_vals
}

In [13]:
fn find_integer_partition(input_num: u32) -> u32 {
    
    let recursive_integers = (0..input_num).collect::<Vec<u32>>();
    
    let store_partition_vals: HashMap<u32, u32> = build_base_partition_vals(recursive_integers);
    
    let all_partition_vals = recursive_partition_function(input_num, &store_partition_vals);
    
    *all_partition_vals.get(&input_num).unwrap()
}

In [14]:
let n = 100;
let soln = find_integer_partition(n);
println!("p({}) = {:?}", n, soln);
println!("No. of different ways {} can be written as a sum of at least 2 positive integers is {}", n, soln - 1);

p(100) = 190569292
No. of different ways 100 can be written as a sum of at least 2 positive integers is 190569291



---


# Problem 78 - Coin Partitions

Let $p(n)$ represent the number of different ways in which $n$ coins can be separated into piles. 
For example, five coins can be separated into piles in exactly seven different ways, so $p(5)=7$.

<div class="margin_left">
OOOOO<br />
OOOO   O<br />
OOO   OO<br />
OOO   O   O<br />
OO   OO   O<br />
OO   O   O   O<br />
O   O   O   O   O
</div>

Find the least value of $n$ for which $p(n)$ is divisible by one million.

---


## Theory

### Approximation Formula

From [wikipedia](https://en.wikipedia.org/wiki/Partition_(number_theory)), the growth rate of $p(n)$ can br approximated by the function,
$$
    \log{p(n)} \approx C \sqrt{n} \quad \text{as } n \to \infty
$$
where $C = \pi \sqrt{\frac{2}{3}}$.

### Application to problem

The above equation can be rearranged to the form, 
$$
    n \approx \frac{3}{2} \Big( \frac{1}{\pi} \log{p(n)} \Big)^2
$$

For this problem, $p(n) = 10^6$ can be substituited to obtain an approximate of n.

In [15]:
use std::f64::consts::PI;
fn approximate_n(target_partition_val: u32) -> f64 {
    let target_val = target_partition_val as f64;
    let inside_bracket = (1.0/PI)*(target_val.log10());
    1.5*inside_bracket*inside_bracket
}

In [16]:
let minimum_pn = 1e12 as u32;
let approx_n = approximate_n(minimum_pn);
println!("Approx n: {}", approx_n);

Approx n: 14.10298405823843


### Misunderstood problem

I think it wants where it is perfectly divisible by 1,000,000 rather than when it exceeds a million.

#### Structure pentagonal numbers

To leverage the functional programming syntax and to reduce repeated code, place both numbers into an array for easy iteration over values.

In [17]:
fn pentagonal_nums(num: u32) -> [i32; 2] {
    [higher_pentagonal_num(num) as i32, lower_pentagonal_num(num) as i32]
}

#### Prevent integer overflow

As $p(n)$ grows exponentially, integer overflow will occur quite quickly. 
Since the problem requires finding when $p(n)\mod{x}$ where $x$ is a million, only the last 6 decimal digits are of concern.
Similarly, if $x=55$, then the last 2 decimal digits are of concern. 
Therefore, the value stored is, 
$$
    p(n) = 
      \begin{cases}
        p(n), & n \leq y \\
        p(n) \mod{x} , & n > y
      \end{cases}
$$
where $y$ is the closest multiple of $10$ that retains the required information

In [18]:
fn next_multiple_10(divisible_by: u32) -> u32 {
    
    if divisible_by % 10 == 0 {
        return divisible_by;
    }
    
    let mut multiple = 10;
    
    loop {
        if divisible_by/multiple != 0 {
            multiple *= 10;
        } else {
            return multiple;
        }
    }
}

#### Solving problem 

The above implementation has been adapted to,
1. Use a vector instead of a HashMap 
2. Loop and terminate based on condition instead of by a counter value

In [19]:
fn find_parition_divisible(divisible_by: u32) -> u32
{
    let mut store_previous: Vec<i32> = Vec::with_capacity(5000);
    let mut partition_integer = 2;
    
    store_previous.push(1);
    store_previous.push(1);
    
    let divisor = next_multiple_10(divisible_by);
    
    loop {
        let max_k = maximum_k_val(partition_integer);
        
        let mut sum: i32 = 0;
        
        for k in 1..=max_k {
            let pent_num = pentagonal_nums(k);
            let this_k = pent_num.iter()
                                .fold(0_i32, 
                                    |sum, val| 
                                    if partition_integer as i32 - val < 0 {
                                        sum
                                    } else {
                                        sum + 
                                        *store_previous.get((partition_integer - *val as u32) as usize)
                                        .unwrap() as i32
            });
            
            if (k+1) & 1 == 0 {
                sum += this_k;
            } else {
                sum -= this_k;
            }
        }
        
        
        let modulo_by_divisible = sum % divisible_by as i32;
        
        if modulo_by_divisible == 0 {
            store_previous.push(sum);
            break
        } else if sum as u32 > divisor {
            store_previous.push(modulo_by_divisible)
        } else {
            store_previous.push(sum)
        }
        
        partition_integer += 1;
        
    }
    
    partition_integer
}

In [20]:
let multiple_of = 1000000;
let soln = find_parition_divisible(multiple_of);
println!("p({}) % {} = 0", soln, multiple_of);

p(55374) % 1000000 = 0
